In [9]:
import cv2
import numpy as np
from pathlib import Path

root_fight = Path("/Users/mukuldixit/desktop/Projects/smartEye/data/raw/RWF-2000/train/Fight")
root_non = Path("/Users/mukuldixit/desktop/Projects/smartEye/data/raw/RWF-2000/train/NonFight")

def get_video_files(path):
    extensions = ['*.mp4', '*.avi', '*.MP4', '*.AVI']
    files = []
    for ext in extensions:
        files.extend(list(path.glob(ext)))
    return sorted(files)

fight_files = get_video_files(root_fight)[:25]
non_fight_files = get_video_files(root_non)[:25]

x, y = [], []
# classes = {"Fight":1, "NonFight":0}

# Ziped them to iterate
for f_path, nf_path in zip(fight_files, non_fight_files):
    
    # 1. Process Fight Video
    cap_f = cv2.VideoCapture(str(f_path))
    mid_f = int(cap_f.get(cv2.CAP_PROP_FRAME_COUNT)) // 2
    cap_f.set(cv2.CAP_PROP_POS_FRAMES, mid_f)
    ret_f, frame_f = cap_f.read()
    cap_f.release()

    # 2. Process Non-Fight Video
    cap_nf = cv2.VideoCapture(str(nf_path))
    mid_nf = int(cap_nf.get(cv2.CAP_PROP_FRAME_COUNT)) // 2
    cap_nf.set(cv2.CAP_PROP_POS_FRAMES, mid_nf)
    ret_nf, frame_nf = cap_nf.read()
    cap_nf.release()

    # 3. Validation and Pre-processing
    if ret_f and ret_nf:
        frame_f = cv2.resize(frame_f, (128, 128)) / 255.0
        frame_nf = cv2.resize(frame_nf, (128, 128)) / 255.0
        x.append(frame_f)
        x.append(frame_nf)
        y.append(1)
        y.append(0)

        # Next step: Append these to a list or save them to a numpy array (.npy)
x = np.array(x).astype("float32")
y = np.array(y)

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

/Users/mukuldixit/Desktop/projects/SmartEye/tf_env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │     3,686,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,705,921 (14.14 MB)

 Trainable params: 3,705,921 (14.14 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(
    x,
    y,
    epochs=10,
    batch_size=4
)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.4600 - loss: 0.9437
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6200 - loss: 0.6542
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7800 - loss: 0.5590
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8200 - loss: 0.3585
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9000 - loss: 0.2823
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9200 - loss: 0.2257
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9000 - loss: 0.2391
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9000 - loss: 0.2435
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.9200 - loss: 0.1745
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9000 - loss: 0.1324
